This is a quick code starter to evaluate sentence similarity

In [1]:
import pandas as pd
import numpy as np
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktTrainer
from nltk import sent_tokenize
import random

# Load the Evaluation Dataset

In [2]:
df = pd.read_csv("data/sts.csv",delimiter="\t",error_bad_lines=False)
df = df[df.Score > 0];len(df)

FileNotFoundError: File b'data/sts.csv' does not exist

In [ ]:
df.head()

In [ ]:
random.choice(df[["Sent1","Sent2","Score"]].values)

In [27]:
mse = ((np.array([1,1,1]) - np.array([1,1,2])) ** 2).mean(axis=None); mse

0.3333333333333333

In [28]:
def evaluate(y_pred,y):
    return "Average Loss: {}".format(((y - y_pred) ** 2).mean(axis=None))

# Test 1

Our initial benchmark can be assigning 3s to all the pairs of sentence. 

In [29]:
def similarity1(sent1,sent2):
    return 3

In [30]:
df["pred"] = df.apply(lambda row: similarity1(row["Sent1"],row["Sent2"]),axis=1)

In [31]:
df.head()

,Dataset,Domain,Score,Sent1,Sent2,pred
0,STS2012-gold,surprise.OnWN,5.00,render one language in another language,restate (words) from one language into another...,3
1,STS2012-gold,surprise.OnWN,3.25,"nations unified by shared interests, history o...",a group of nations having common interests.,3
2,STS2012-gold,surprise.OnWN,3.25,"convert into absorbable substances, (as if) wi...",soften or disintegrate by means of chemical ac...,3
3,STS2012-gold,surprise.OnWN,4.00,"devote or adapt exclusively to an skill, study...",devote oneself to a special area of work.,3
4,STS2012-gold,surprise.OnWN,3.25,elevated wooden porch of a house,a porch that resembles the deck on a ship.,3


In [32]:
evaluate(df.pred.values,df.Score.values)

'Average Loss: 1.9734321848734255'

# Test 2

Keep on going with other tests...